In [18]:
import numpy as np
import pandas as pd


from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from utils import task_score

In [7]:
x_train = pd.read_csv('../../../data/x_train.txt', header=None, sep=' ')
y_train = pd.read_csv('../../../data/y_train.txt', header=None, sep=' ')
df_train = pd.concat([x_train, y_train], axis=1)
df_train.columns = [ 'x'+str(i) for i in range(1, df_train.shape[1]) ] + ['y']    

In [19]:
initial_features = [
    "x1",
    "x10",
    "x101",
    "x102",
    "x103",
    "x104",
    "x105",
    "x106",
    "x132",
    "x140",
    "x149",
    "x153",
    "x156",
    "x176",
    "x191",
    "x2",
    "x22",
    "x221",
    "x229",
    "x253",
    "x286",
    "x3",
    "x304",
    "x322",
    "x323",
    "x324",
    "x329",
    "x336",
    "x35",
    "x352",
    "x36",
    "x4",
    "x40",
    "x404",
    "x413",
    "x423",
    "x459",
    "x463",
    "x499",
    "x5",
    "x58",
    "x6",
    "x65",
    "x7",
    "x74",
    "x8",
    "x81",
    "x9",
    "x99",
]

df_train = df_train[initial_features + ["y"]]
X_train, X_test, y_train, y_test = train_test_split(
    df_train.drop(columns="y"), df_train["y"], test_size=0.2, stratify=df_train["y"], random_state=42
)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
lasso = LassoCV(cv=5, random_state=42 ,n_jobs=-1, n_alphas=100)
lasso.fit(X_train, y_train)
coef = lasso.coef_
print(np.sum(coef!= 0)) # definitely too many features
features = df_train.columns[np.where(lasso.coef_ != 0)]
print(features)


42
Index(['x1', 'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x132', 'x140',
       'x149', 'x153', 'x156', 'x176', 'x191', 'x22', 'x221', 'x229', 'x253',
       'x286', 'x304', 'x322', 'x323', 'x324', 'x329', 'x336', 'x35', 'x352',
       'x36', 'x40', 'x404', 'x413', 'x423', 'x459', 'x463', 'x499', 'x5',
       'x58', 'x65', 'x74', 'x8', 'x81', 'x99'],
      dtype='object')


In [43]:
X_train_ = X_train[:,coef!= 0]
X_test_ = X_test[:,coef!= 0]

In [51]:
svc = SVC(kernel='rbf', random_state=42,probability=True)
svc.fit(X_train_, y_train)
print(task_score(svc,X_test_, y_test))
# just bad

-1350.0


---
SVM L1

In [61]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

In [69]:
clf = LinearSVC(C=0.005, penalty="l1", dual=False, random_state=42)
selection = SelectFromModel(clf).fit(X_train, y_train).get_support()
print(np.sum(selection)) # definitely too many features
features = df_train.columns[np.where(selection)]
print(features)

20
Index(['x106', 'x140', 'x153', 'x156', 'x176', 'x22', 'x221', 'x253', 'x324',
       'x329', 'x336', 'x352', 'x36', 'x404', 'x413', 'x459', 'x499', 'x58',
       'x65', 'x81'],
      dtype='object')


In [70]:
X_train_ = X_train[:,selection]
X_test_ = X_test[:,selection]

In [71]:
svc = SVC(kernel='rbf', random_state=42,probability=True)
svc.fit(X_train_, y_train)
print(task_score(svc,X_test_, y_test))
# also very bad

2150.0
